In [ ]:
import ee
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import seaborn as sns

# Utility Functions
# ***********************

# Input coordinates to make a geometry

def makeGeometry(coordinates):

  dlat = 0.326
  dlong = 0.21568

  lat = coordinates[0]
  longit = coordinates[1]
  latp = lat + dlat
  longp = longit + dlong

  print(lat, longit, latp, longp)
  geometry = ee.Geometry.Rectangle(lat, longit, latp, longp)

  return geometry


# ***********************
# Sentinel
# ***********************

# Function to apply scale factor to convert
# pixel values to reflectances
def scaleBands(image):
  return image.multiply(0.0001) \
    .copyProperties(image, ['system:time_start'])


# Function to mask pixels with low CS+ QA scores.
def maskLowQA(image):
  qaBand = 'cs'
  clearThreshold = 0.5
  mask = image.select(qaBand).gte(clearThreshold)
  return image.updateMask(mask)


# Function to compute spectral indices
def addIndices(image):
  ndvi = image.normalizedDifference(['B8', 'B4']) \
    .rename('ndvi')

  mndwi = image.normalizedDifference(['B3', 'B11']) \
    .rename('mndwi')

  ndbi = image.normalizedDifference(['B11', 'B8']) \
    .rename('ndbi')

  evi = image.expression(
    '2.5 * ((NIR - RED)/(NIR + 6*RED - 7.5*BLUE + 1))', {
      'NIR': image.select('B8'),
      'RED': image.select('B4'),
      'BLUE': image.select('B2')
    }).rename('evi')

  bsi = image.expression(
      '(( X + Y ) - (A + B)) /(( X + Y ) + (A + B)) ', {
        'X': image.select('B11'),
        'Y': image.select('B4'),
        'A': image.select('B8'),
        'B': image.select('B2'),
    }).rename('bsi')

  return image \
    .addBands(ndvi) \
    .addBands(mndwi) \
    .addBands(ndbi) \
    .addBands(evi) \
    .addBands(bsi)


# Preparing Sentinel-2 composite

def s2CompositePrep(s2, startDate, endDate, gridProjection, geometry):
  filteredS2 = s2 \
    .filter(ee.Filter.date(startDate, endDate)) \
    .filter(ee.Filter.bounds(geometry))

  # Extract the projection before any processing
  s2Projection = ee.Image(filteredS2.first()).select('B4').projection()


  # Use Cloud Score+ cloud mask
  csPlus = ee.ImageCollection(
    'GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')
  csPlusBands = csPlus.first().bandNames()

  # We need to add Cloud Score + bands to each Sentinel-2
  # image in the collection
  # This is done using the linkCollection() function
  filteredS2WithCs = filteredS2.linkCollection(
    csPlus, csPlusBands)

  # Apply all the pre-processing steps
  # Order in which the functions are applied is important
  s2Processed = filteredS2WithCs \
    .map(maskLowQA) \
    .select('B.*') \
    .map(scaleBands) \
    .map(addIndices)

  # Create the S2 composite
  s2Composite = s2Processed.median() \
    .setDefaultProjection(gridProjection)

  return s2Composite


# s2 projection

def getS2projection(s2, startDate, endDate, geometry):
  filteredS2 = s2 \
    .filter(ee.Filter.date(startDate, endDate)) \
    .filter(ee.Filter.bounds(geometry))

  # Extract the projection before any processing
  s2Projection = ee.Image(filteredS2.first()).select('B4').projection()

  return s2Projection



# *********************
# CEDA CCI biomass
# *********************

def cciPrep(biomass_cci, startDate, endDate, gridProjection, geometry):
  biomass_cci_filtered = biomass_cci \
    .filter(ee.Filter.date(startDate, endDate)) \
    .filter(ee.Filter.bounds(geometry))

  cciProjection = ee.Image(biomass_cci.first()) \
    .select('AGB').projection()

  cciMosaic = biomass_cci_filtered.mosaic() \
    .setDefaultProjection(gridProjection)


  return cciMosaic


# CCI projection

def getCCIprojection(biomass_cci, startDate, endDate, geometry):
  biomass_cci_filtered = biomass_cci \
    .filter(ee.Filter.date(startDate, endDate)) \
    .filter(ee.Filter.bounds(geometry))

  cciProjection = ee.Image(biomass_cci.first()) \
    .select('AGB').projection()

  return cciProjection



# ************
# Export data
# ************

# Resample to a Grid

def stack(array, array_len, gridProjection):

  # Create a stacked image
  # We assemble a composite with all the bands

  stacked = array[0]
  for i in range(1, array_len):
    stacked = stacked.addBands(array[i])

  #  Set the resampling mode
  stacked = stacked.resample('bilinear')

  # Aggregate pixels with 'mean' statistics
  stackedResampled = stacked \
    .reduceResolution(**{
      'reducer': ee.Reducer.mean(),
      'maxPixels': 1024
    }) \
    .reproject(**{
    'crs': gridProjection
    })

  return stackedResampled


# Convert the satellite images
# and CEDA dataset to a Pandas dataframe

def makedfCEDA(stackedCEDA, geometry, gridScale):

  training = stackedCEDA.sample(**{
      #'numPoints': numSamples,
      'region': geometry,
      'scale': gridScale,
      'tileScale': 16,
    })

  df = ee.data.computeFeatures({
    'expression': training,
    'fileFormat': 'PANDAS_DATAFRAME'
  })

  return df


# Convert the satellite images and
# CEDA dataset to a CSV file

def makeCSVCEDA(stackedCEDA, geometry, gridScale, folder, fileName):

  training = stackedCEDA.sample(**{
      #'numPoints': numSamples,
      'region': geometry,
      'scale': gridScale,
      'tileScale': 16,
    })

  task = ee.batch.Export.table.toDrive(**{
  'collection': training,
  'description': fileName,
  'fileFormat': 'CSV',
  'folder': folder
  })

  task.start()


# ************
# Models
# ************

# Use models to predict biomass
def modelPred(dfC):

  models = [
    LinearRegression(),
   # DecisionTreeRegressor(random_state=0),
   # RandomForestRegressor(max_depth=2, random_state=0),
    xgb.XGBRegressor()
  ]

  modelNames = [
    'Linear Regression',
    #'Decission Tree',
    #'Random Forest',
    'XGBoost'
  ]

  nmodels = len(models)

  dfC = dfC.dropna()

  x = dfC
  y = dfC[['AGB']]

  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

  x_train = x_train.drop(['AGB','SD', 'pSD'], axis=1)
  x_test = x_test.drop(['AGB','SD', 'pSD'], axis=1)


  y_predAr = []
  r2Ar = []
  rmseAr = []

  for j in range(nmodels):
    if modelNames[j] == 'Random Forest':
      fitModel = models[j].fit(x_train, y_train.values.ravel())
    else:
      fitModel = models[j].fit(x_train, y_train)

    y_pred = fitModel.predict(x_test)
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared = False)

    y_predAr.append(y_pred)
    r2Ar.append(r2)
    rmseAr.append(rmse)

  pred = [y_test, y_predAr, r2Ar, rmseAr, modelNames]

  return pred



# Use models to predict biomass, weight data according to 1/SD

def modelPredErr(dfC):

  models = [
    LinearRegression(),
#    DecisionTreeRegressor(random_state=0),
#    RandomForestRegressor(max_depth=2, random_state=0),
    xgb.XGBRegressor()
  ]

  modelNames = [
    'Linear Regression',
 #   'Decission Tree',
 #   'Random Forest',
    'XGBoost'
  ]

  nmodels = len(models)

  dfC = dfC.dropna()

  x = dfC
  y = dfC[['AGB']]

  x_train, x_test, y_train, y_test  = train_test_split(x, y, test_size=0.33)

  weights = 1/x_train[['SD']]

  x_train = x_train.drop(['AGB','SD', 'pSD'], axis=1)
  x_test = x_test.drop(['AGB','SD', 'pSD'], axis=1)


  y_predAr = []
  r2Ar = []
  rmseAr = []

  for j in range(nmodels):
    if modelNames[j] == 'Random Forest':
      fitModel = models[j].fit(x_train, y_train.values.ravel(), sample_weight=weights.values.ravel())
    else:
      fitModel = models[j].fit(x_train, y_train, sample_weight=weights.values.ravel())

    y_pred = fitModel.predict(x_test)
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared = False)

    y_predAr.append(y_pred)
    r2Ar.append(r2)
    rmseAr.append(rmse)

  pred = [y_test, y_predAr, scoreAr, modelNames]

  return pred

# ***********************************
# Reading and writing model / EDA
# results to a file
# ***********************************

# EDA on a column
def EDAcol(col):

  q_ar = [0.05, 0.25, 0.50, 0.75, 0.95]
  statNames = ['q05', 'q025', 'q50', 'q75', 'q95', 'rangeL', 'rangeS', 'mean', 'var']

  nq = len(q_ar)
  statVals = []

  for i in range(0, nq):
    quant = col.quantile(q_ar[i])
    statVals.append(round(quant,4))

  rangeL = statVals[nq-1] - statVals[0]
  rangeS = statVals[nq-2] - statVals[1]
  mean = col.mean()
  var = col.var()

  statVals.append(round(rangeL,4))
  statVals.append(round(rangeS,4))
  statVals.append(round(mean,4))
  statVals.append(round(var,6))

  return statNames, statVals

# EDA on a dataframe
def EDAdf(df):
  colNames = list(df.columns)
  ncol = len(colNames)

  statValsAll=[]
  statNames=[]

  for i in range(0, ncol):
    statNames, statVals = EDAcol(df[colNames[i]])
    statValsAll.append(statVals)

  statValsAll = pd.DataFrame(np.array(statValsAll).T, columns=colNames, index = statNames)

  return statValsAll

 # Write model results to a file
def modelResultsToDF(rsq, rmse, modelNames):
  names=['rsq', 'rmse']

  modelResults=[rsq, rmse]
  modelResults = pd.DataFrame(np.array(modelResults).T, columns=names, index = modelNames)
  modelResults = modelResults.transpose()

  return modelResults

# Reading model and EDA results from a file
# and storing them in a dataframe
def EDAModelFilestoDF(filename):

  df = pd.read_csv(filename)
  df.rename(columns={df.columns[0]: 'rows'}, inplace=True)
  df = df.set_index(df.iloc[:,0])
  df = df.drop(['rows'], axis =1)

  return df

########################### Plottting Functions

# clean up dataframe before analysis
def cleanDF(filename):
  dfC = pd.read_csv(filename)
  dfC = dfC.drop(['.geo'], axis=1)
  dfC = dfC.drop(['system:index'], axis=1)

  # drop missing values
  dfC = dfC.dropna()

  # add percentage SD column
  dfC.loc[dfC['AGB'] == 0, ['AGB']] = 0.001
  dfC.loc[dfC['SD'] == 0, ['SD']] = 0.001

  dfC['pSD'] = dfC['SD']/dfC['AGB']

  # only keep points with percentage error < 1
  dfC = dfC[dfC.pSD < 1]

  # sampling points for scatter plot to see AGB trend with other variables
  if dfC.shape[0] > 1000:
    dfCS = dfC.sample(n=1000, random_state=0)
  else:
    dfCS = dfC

  return dfC, dfCS

# EDA Plots
def EDAplotsAll(placeNameProjectionType, dfC, dfCS):
  n_rA = 5 # n_rows
  n_cA = 4

  props = dict(boxstyle='round, pad=1', facecolor='white')

  figTA, axesTA = plt.subplots(n_rA, n_cA) # trends all
  figHA, axesHA = plt.subplots(n_rA, n_cA)  # hist all

  colName = list(dfC.columns)
  df_col_len = len(colName)

  for row_num in range(0, n_rA):
    for col_num in range(0, n_cA):
      df_col = n_cA*row_num + col_num
      # scatter plot
      loqS = dfCS.iloc[:, df_col].quantile(0.05)
      hiqS = dfCS.iloc[:, df_col].quantile(0.95)
      axAGBA = axesTA[row_num][col_num]
      axAGBA.scatter(dfCS.iloc[:, df_col], dfCS['AGB'], s=2, marker='o')
      axAGBA.set_title('AGB vs '+ colName[df_col])
      axAGBA.set_xlim([loqS, hiqS])
      # histogram
      axHistA = axesHA[row_num][col_num]
      loq = dfC.iloc[:, df_col].quantile(0.05)
      hiq = dfC.iloc[:, df_col].quantile(0.95)
      q25 = dfC.iloc[:, df_col].quantile(0.25)
      q75 = dfC.iloc[:, df_col].quantile(0.75)
      range90 = hiq - loq
      range50 = q75 - q25
      mean = dfC.iloc[:, df_col].mean()

      textstr = 'mean ' + round(mean, 3).astype(str) + '\n' +'large ' + round(range90, 3).astype(str) + '\n' + 'small '  + round(range50, 3).astype(str)

      col = dfC[dfC.iloc[:, df_col].between(loq, hiq)].iloc[:, df_col]
      axHistA.hist(
          col,
          bins=10, # Number of bins
          edgecolor='black', # Color of the border
          color='g',
          rwidth=0.8
        )
      axHistA.grid()
      axHistA.locator_params(axis='x', nbins=5)
      axHistA.set_title(colName[df_col])
      hpos=0.7
      if colName[df_col] == 'evi' or colName[df_col] == 'ndvi':
        hpos=0.05
      axHistA.text(hpos, 0.90, textstr, transform=axHistA.transAxes, fontsize=6, verticalalignment='top', bbox=props)

      figTA.set_size_inches(15, 10)
      figTA.suptitle(placeNameProjectionType, fontsize=12)
      figTA.tight_layout(pad=1)

      figHA.set_size_inches(15, 10)
      figHA.suptitle(placeNameProjectionType, fontsize=12)
      figHA.tight_layout(pad=1)

  return figTA, figHA

def EDAplotsL(placeNameProjectionType, dfC, dfCS):

  lesscol = ['AGB', 'B1', 'B8', 'evi', 'ndvi', 'pSD']
  props = dict(boxstyle='round , pad=1', facecolor='white')

  n_rL = 2 # n_rows less
  n_cL = 3
  figTL, axesTL = plt.subplots(n_rL, n_cL) # trends less
  figHL, axesHL = plt.subplots(n_rL, n_cL) # hist less

  for row_num in range(0, n_rL):
    for col_num in range(0, n_cL):
      df_col = n_cL*row_num + col_num
      # scatter plot
      loqS = dfCS[lesscol[df_col]].quantile(0.05)
      hiqS = dfCS[lesscol[df_col]].quantile(0.95)
      axAGBL = axesTL[row_num][col_num]
      axAGBL.scatter(dfCS[lesscol[df_col]], dfCS['AGB'], s=3, marker='o')
      axAGBL.set_title('AGB vs '+ lesscol[df_col])
      axAGBL.set_xlim([loqS, hiqS])
      # histogram
      axHistL = axesHL[row_num][col_num]
      loq = dfC[lesscol[df_col]].quantile(0.05)
      hiq = dfC[lesscol[df_col]].quantile(0.95)
      q25 = dfC[lesscol[df_col]].quantile(0.25)
      q75 = dfC[lesscol[df_col]].quantile(0.75)
      range90 = hiq - loq
      range50 = q75 - q25
      mean = dfC[lesscol[df_col]].mean()

      textstr = 'mean ' + round(mean, 3).astype(str) + '\n\n' +'large ' + round(range90, 3).astype(str) + '\n\n' + 'small '  + round(range50, 3).astype(str)
      col = dfC[dfC[lesscol[df_col]].between(loq, hiq)][lesscol[df_col]]
      axHistL.hist(
          col,
          bins=10, # Number of bins
          edgecolor='black', # Color of the border
          color='g',
          rwidth=0.8
        )
      axHistL.grid()
      axHistL.locator_params(axis='x', nbins=5)
      axHistL.set_title(lesscol[df_col])
      hpos=0.7
      if lesscol[df_col] == 'evi' or lesscol[df_col] == 'ndvi':
        hpos=0.05
      axHistL.text(hpos, 0.95, textstr, transform=axHistL.transAxes, fontsize=10, verticalalignment='top', bbox=props)

  figHL.set_size_inches(15, 10)
  figHL.suptitle(placeNameProjectionType, fontsize=12)
  figHL.tight_layout(pad=1)

  figTL.set_size_inches(15, 10)
  figTL.suptitle(placeNameProjectionType, fontsize=12)
  figTL.tight_layout(pad=1)

  return figTL, figHL

# Correlation plots
def corrPlots(placeNameProjectionType, dfC):
  lesscol = ['AGB', 'B1', 'B8', 'evi', 'ndvi', 'pSD']

  figCorr = plt.figure(figsize=(16, 10))
  heatmap = sns.heatmap(dfC.corr(), vmin=-1, vmax=1, annot=True, cmap="viridis_r")
  heatmap.set_title(placeNameProjectionType, fontdict={'fontsize':18}, pad=12)

  figCorrL = plt.figure(figsize=(16, 10))
  heatmapL = sns.heatmap(dfC[lesscol].corr(), vmin=-1, vmax=1, annot=True, cmap="viridis_r")
  heatmapL.set_title(placeNameProjectionType, fontdict={'fontsize':18}, pad=12)

  return figCorr, figCorrL

# Plot model results
def modelPlots(placeNameProjectionType, pred):
  y_test = pred[0]
  y_predAr = pred[1]
  r2Ar = pred[2]
  rmseAr = pred[3]
  modelNames = pred[4]

  # Plots of model results
  props = dict(boxstyle='round', facecolor='white')

  n_cols = 2

  figModels, axesModels = plt.subplots(1, n_cols)

  for iModel in range(0, n_cols):
    axModels = axesModels[iModel]
    axModels.scatter(y_test, y_predAr[iModel], s=2, marker='o')
    axModels.set_title(modelNames[iModel]) #+ ' Rsq ' + round(r2Ar[iModel], 2).astype(str) + ' RMSE ' + round(rmseAr[iModel], 2).astype(str))
    textstr = 'Rsq ' + round(r2Ar[iModel], 3).astype(str) + '\n\n' +'RMSE ' + round(rmseAr[iModel], 3).astype(str)
    axModels.text(0.05, 0.95, textstr, transform=axModels.transAxes, fontsize=12, verticalalignment='top')

  figModels.set_size_inches(16, 8)
  figModels.suptitle('y_pred vs y_test ' + placeNameProjectionType, fontsize=12)
  figModels.tight_layout(pad=2)

  return figModels